# Data Exploration - Price Data

Interactive exploration and validation of collected price data.

**Goals:**
- Validate data quality
- Explore price movements and patterns
- Test basic calculations (returns, volatility)
- Identify any data issues

In [1]:
# Setup
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent))

import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("Imports successful!")

Imports successful!


## 1. Load Data

In [38]:
# Load the full dataset
data_path = Path('../data/price/daily/sample_universe_2022-10-24_to_2025-10-23.parquet')
df = pl.read_parquet(data_path)

print(f"Loaded {len(df):,} rows")
print(f"Columns: {df.columns}")
print(f"\nShape: {df.shape}")
df.head()

Loaded 15,040 rows
Columns: ['symbol', 'date', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'dividends', 'stock_splits']

Shape: (15040, 10)


symbol,date,open,high,low,close,adj_close,volume,dividends,stock_splits
str,date,f64,f64,f64,f64,f64,i64,f64,f64
"""TSLA""",2022-10-24,205.820007,213.5,198.589996,211.25,211.25,100446800,0.0,0.0
"""GOOGL""",2022-10-24,101.800003,102.75,99.980003,102.519997,101.817749,27176400,0.0,0.0
"""META""",2022-10-24,127.25,133.479996,124.57,129.720001,128.925232,63563400,0.0,0.0
"""NFLX""",2022-10-24,290.230011,290.48999,280.359985,282.450012,282.450012,13326400,0.0,0.0
"""JPM""",2022-10-24,122.07,123.099998,121.330002,122.379997,113.659988,12624200,0.0,0.0


In [45]:
df = df.with_columns([
    pl.col("date").dt.to_string("%Y-%m-%d").alias("ds"),
    ((pl.col("close")-pl.col("open"))/pl.col("open")).alias("y_true"),
    ((pl.col("high")-pl.col("open"))/pl.col("open")).alias("y_pred")
    ])

In [46]:
test = df.filter(pl.col("ds")=="2022-10-24").select(["y_true", "y_pred"])

In [51]:
test = test.with_columns([
    pl.col("y_true").rank().alias("true_rank"),
    pl.col("y_pred").rank().alias("pred_rank")
]).with_columns(
    pl.col("pred_rank").sub(pl.col("true_rank")).alias("rank_error")
)


In [55]:
test.select([
    pl.col("rank_error").mean().alias("mean_error"),
    pl.col("rank_error").abs().mean().alias("mae"),
    pl.col("rank_error").pow(2).mean().alias("mse"),
    pl.col("rank_error").pow(2).mean().sqrt().alias("rmse"),
])

mean_error,mae,mse,rmse
f64,f64,f64,f64
0.0,1.8,5.2,2.280351


## 2. Data Quality Checks

In [ ]:
# Basic statistics
print("=== DATA SUMMARY ===")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"Number of symbols: {df['symbol'].n_unique()}")
print(f"\nSymbols: {sorted(df['symbol'].unique().to_list())}")

# Check for missing values
print("\n=== MISSING VALUES ===")
null_counts = df.null_count()
print(null_counts)

# Rows per symbol
print("\n=== ROWS PER SYMBOL ===")
rows_per_symbol = df.group_by('symbol').agg(pl.len().alias('count')).sort('count', descending=True)
print(rows_per_symbol)

In [ ]:
# Check for negative prices (should be none)
price_cols = ['open', 'high', 'low', 'close', 'adj_close']
for col in price_cols:
    neg_count = df.filter(pl.col(col) < 0).height
    if neg_count > 0:
        print(f"WARNING: {neg_count} negative values in {col}")
    else:
        print(f"✓ {col}: No negative values")

# Check high >= low
invalid_hl = df.filter(pl.col('high') < pl.col('low')).height
print(f"\n{'✓' if invalid_hl == 0 else '✗'} High >= Low: {invalid_hl} violations")

## 3. Price Movement Analysis

In [ ]:
# Calculate basic metrics
df_metrics = df.with_columns([
    # Daily return using adj_close
    (pl.col('adj_close').pct_change().over('symbol')).alias('daily_return'),
    # Daily range
    ((pl.col('high') - pl.col('low')) / pl.col('close')).alias('daily_range_pct'),
    # Volume in millions
    (pl.col('volume') / 1_000_000).alias('volume_millions')
])

print("Metrics calculated!")
df_metrics.select(['symbol', 'date', 'adj_close', 'daily_return', 'daily_range_pct']).head(10)

In [ ]:
# Summary statistics by symbol
summary = df_metrics.group_by('symbol').agg([
    pl.col('daily_return').mean().alias('avg_daily_return'),
    pl.col('daily_return').std().alias('volatility'),
    pl.col('volume_millions').mean().alias('avg_volume_M'),
    pl.col('adj_close').min().alias('min_price'),
    pl.col('adj_close').max().alias('max_price'),
]).sort('volatility', descending=True)

print("\n=== SUMMARY BY SYMBOL ===")
print(summary)

## 4. Visualizations

In [ ]:
# Plot price evolution for a few stocks
sample_symbols = ['AAPL', 'MSFT', 'NVDA', 'TSLA']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
axes = axes.flatten()

for idx, symbol in enumerate(sample_symbols):
    symbol_data = df_metrics.filter(pl.col('symbol') == symbol).sort('date')
    
    ax = axes[idx]
    ax.plot(symbol_data['date'].to_pandas(), symbol_data['adj_close'].to_pandas(), linewidth=1.5)
    ax.set_title(f'{symbol} - Adjusted Close Price', fontsize=12, fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price ($)')
    ax.grid(True, alpha=0.3)
    
plt.tight_layout()
plt.show()

In [ ]:
# Daily returns distribution
fig, ax = plt.subplots(figsize=(12, 6))

for symbol in sample_symbols:
    symbol_data = df_metrics.filter(pl.col('symbol') == symbol)
    returns = symbol_data['daily_return'].drop_nulls().to_pandas()
    ax.hist(returns, bins=50, alpha=0.5, label=symbol)

ax.set_title('Daily Returns Distribution', fontsize=14, fontweight='bold')
ax.set_xlabel('Daily Return')
ax.set_ylabel('Frequency')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Volatility comparison (last 60 days rolling)
fig, ax = plt.subplots(figsize=(14, 6))

for symbol in sample_symbols:
    symbol_data = df_metrics.filter(pl.col('symbol') == symbol).sort('date')
    
    # Calculate 60-day rolling volatility
    volatility = (
        symbol_data
        .select([
            'date',
            pl.col('daily_return').rolling_std(window_size=60).alias('volatility_60d')
        ])
    )
    
    ax.plot(volatility['date'].to_pandas(), volatility['volatility_60d'].to_pandas(), label=symbol, linewidth=2)

ax.set_title('60-Day Rolling Volatility', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Volatility (Std Dev of Returns)')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 5. Adjusted vs Unadjusted Close

In [ ]:
# Check for splits and dividends
splits = df.filter(pl.col('stock_splits') > 0)
dividends = df.filter(pl.col('dividends') > 0)

print(f"Stock splits: {splits.height} events")
print(f"Dividends: {dividends.height} events")

if splits.height > 0:
    print("\nStock split events:")
    print(splits.select(['symbol', 'date', 'stock_splits', 'close', 'adj_close']))

if dividends.height > 0:
    print("\nSample dividend events:")
    print(dividends.select(['symbol', 'date', 'dividends', 'close', 'adj_close']).head(10))

In [ ]:
# Calculate adjustment ratio
df_adj = df.with_columns([
    ((pl.col('adj_close') / pl.col('close')) - 1.0).alias('adjustment_ratio')
])

# Find significant adjustments
significant_adj = df_adj.filter(pl.col('adjustment_ratio').abs() > 0.01)
print(f"\nRows with >1% adjustment: {significant_adj.height}")
if significant_adj.height > 0:
    print(significant_adj.select(['symbol', 'date', 'close', 'adj_close', 'adjustment_ratio', 'dividends', 'stock_splits']).head(20))

## 6. Custom Analysis Space

Use this section for ad-hoc exploration

In [ ]:
# Your custom analysis here
# Example: Calculate 30-day forward returns

df_forward = df.sort(['symbol', 'date']).with_columns([
    # 30-day forward return
    ((pl.col('adj_close').shift(-20) / pl.col('adj_close')) - 1.0)
    .over('symbol')
    .alias('forward_20d_return')
])

# Show distribution of forward returns
print("30-day forward return statistics:")
print(df_forward['forward_20d_return'].describe())

In [ ]:
# Save any processed data if needed
# df_processed.write_parquet('../data/processed/my_analysis.parquet')